In [3]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import pandas as pd


class chdtv_news:
    
    def __init__(self, keyword, result = []):
        
        self.keyword = keyword
        self.result = result
    
    def get_chdtv_news(self, date):
        
        starttime = datetime.datetime.now()
        print(starttime)

        url = "https://www.chinatimes.com/search/{}?chdtv".format(self.keyword)
        #爬取特定頁數
        #url ="https://www.chinatimes.com/search/{}?page=2&chdtv".format(self.keyword)

        headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "cache-control": "max-age=0",
        "referer": "https://www.chinatimes.com/?chdtv",
        "sec-ch-ua-mobile": "?0",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"

        }
        while True:

            res = requests.get(url,headers = headers)

            soup = BeautifulSoup(res.text, "html.parser")

            for col in soup.select("div.col"):
                article_date = col.select_one("span.date").text
                article_hour = col.select_one("span.hour").text
                
                try:
                    #try-except防止文章遺失造成程式停止
                    each_article_text= ""                           #建立空字串方便後面加入文章內容
                    article_url = col.select_one("h3 a")["href"]    #取得文章url
                    title = col.select_one("h3 a").text             #取得文章標題



                    print(article_date)
                    print(title)
                    print(article_url)


                    #設定時間區間
                    article_date_str = "".join(article_date.split("/"))
                    if int(article_date_str) < int(date):
                        break

                    else:

                        headers["referer"] = article_url
                        res2 = requests.get(article_url, headers = headers)
                        soup2 = BeautifulSoup(res2.text, "html.parser")

                        each_article = {}  #建立空字典方便接下來暫存統整每篇文章內容

                        paragraphs = soup2.select("div.article-body p")

                        for p in paragraphs:
                            each_article_text += p.text

                        print(each_article_text)
                        each_article_text = each_article_text.strip("(時報資訊)")
                        each_article_text = each_article_text.strip("(工商 )")
                        each_article_text = each_article_text.strip("(中時新聞網)")
                        each_article_text = each_article_text.strip("（袁延壽）") #去除文末的引用資訊

                        #加入關鍵字、標題、日期、時間、網址、內容

                        each_article["關鍵字"] = self.keyword
                        each_article["標題"] = title
                        each_article["日期"] = article_date
                        each_article["時間"] = article_hour
                        each_article["網址"] = article_url
                        each_article["內容"] = each_article_text


                        #將每一篇文章資訊加入list
                        self.result.append(each_article)
                        
                        #以防萬一邊爬邊存 
                        df_new = pd.DataFrame.from_records(self.result)
                        df_new.to_csv("new_chdtv_covid19_news.csv", encoding = "utf-8-sig", index = False)


                        #設定隨機休息時間
                        sleep_time = random.randint(3,10)
                        print("sleep time: %s sec"%(sleep_time))
                        time.sleep(sleep_time)
                        
                except:
                    pass

            try:
                url = "https://www.chinatimes.com" + soup.select("a.page-link")[-2]['href'] + "&chdtv"
                if int(article_date_str) < int(date):
                        break

                else:
                    continue

            except:
                break
                
                
        endtime = datetime.datetime.now()
        print(endtime)
        
        return "finished"
    
    def get_content(self):
        df = pd.DataFrame.from_records(self.result)
        df.to_csv("chdtv_covid19_news.csv", encoding = "utf-8-sig", index = False)
        #columns = [關鍵字,標題,日期,時間,網址,內容] 
        
        return "OK"
    
    def append_content(self):
        df = pd.DataFrame.from_records(self.result)
        df.to_csv("chdtv_covid19_news.csv", mode='a', encoding = "utf-8-sig", index = False)
        
        return "OK"


In [2]:
A = chdtv_news("新冠肺炎")
A.get_chdtv_news("20191201")

2021-01-09 01:27:38.006960
2021/01/08
《熱門族群》去年財報揭曉：台塑、台化、台塑化染疫 南亞逆增
https://www.chinatimes.com/realtimenews/20210108003632-260410
台塑四寶公布2020年財報成績單，受到新冠肺炎衝擊，其中以受原油價格影響最直接的台塑化獲利影響最鉅，全年每股稅後盈餘0.79元，台塑、台化也均相較2019年呈顯明顯衰退，南亞則受益去年第四季本業營業利益攀升，全年每股獲利3.24元，是台塑四寶中唯一相較前年正成長的個股。台塑(1301)2020年本業利益170億元，年衰退15.4%，合併稅前利益240億元，年衰退42.9%，每股稅前盈餘為3.78元，稅後利益199億元，比2019年減少173億元，衰退46.5%，歸屬母公司每股稅後盈餘為3.14元；台塑表示，受新冠肺炎疫情影響，各國封城鎖國導致需求急遽萎縮，原油期貨價格甚至在第2季出現負數，全球經濟嚴重衰退，加上同業新增產能陸續投產，市場供給增加，拖累石化產品價格下跌，所有產品平均價格比2019年低，幅度介於12~38%。南亞(1303)2020年合併稅前利益較2019年增加37億6,062萬元，主要為營業利益212.2億元，較2019年增加113.3億元，大幅成長114.65%：自年初以來，產業面對疫情，由被動因應、設法降低傷害，逐漸轉為積極適應、從中尋求新經濟商機。南亞2020年本業營業利益逐季增加，於第4季達到高峰，全年累計營收雖因油價影響而減少，但獲利卻比2019年顯著成長，歸屬母公司稅後利益為257.01億元，每股稅後盈餘3.24元。台化(1326)2020年全年度歸屬母公司稅後利益為195.4億元，每股稅後盈餘3.34元，比2019年度稅後每股盈餘5.09元，減少1.75元，衰退34.4%；台化表示，去年雖受疫情影響，市場需求驟減，原油及石化產品價格大跌，但第2季起各國陸續解封並重啟經濟，尤其大陸市場恢復明顯，隨著疫後新商機出現，積極拓展銷售，惟整體尚未回復至疫情前水準，營業利益仍較前年減少6.6%。台塑化(6505)2020年自結合併稅前利益87.21億元，比2019年稅前利益448.98億元減少361.76億元，全年自結合併稅後利益74.27億元，比2019年稅後利益367.48億元減少293.2

ConnectionError: HTTPSConnectionPool(host='www.chinatimes.com', port=443): Max retries exceeded with url: /search/%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E?page=2&chdtv (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000201F0610BE0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
#abc = A.get_content()

In [ ]:
b =A.append_content()